# Prakiraan Konsumsi Listrik dengan XGBoost dan Sktime

Notebook ini bertujuan untuk melakukan prakiraan konsumsi listrik menggunakan model XGBoost dengan pendekatan berbasis sktime. Kami akan menggunakan beberapa strategi prakiraan (Direct, DirRec, Recursive, dan Multioutput) dengan dan tanpa fitur eksogen, serta membandingkannya dengan model XGBoost sederhana. Parameter `n_jobs=-1` digunakan untuk memaksimalkan penggunaan core CPU.

## 1. Konfigurasi dan Instalasi Paket
Mengatur lingkungan dengan menginstal pustaka yang diperlukan dan mendefinisikan semua parameter konfigurasi.

In [1]:
# Instalasi paket yang diperlukan
!pip install sktime pandas numpy xgboost openpyxl -q

# Impor pustaka
import pandas as pd
import numpy as np
import xgboost as xgb
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    DirectTabularRegressionForecaster,
    DirRecTabularRegressionForecaster,
    MultioutputTabularRegressionForecaster,
    RecursiveTabularRegressionForecaster,
)
import time
import os

# Konfigurasi
RANGE_FOLD = 2  # Jumlah fold untuk validasi silang
WINDOW_LENGTH = 7 * 3 * 24  # Panjang jendela 3 minggu (dalam jam)
FORECAST_HORIZON = 24  # Horizon prakiraan (24 jam ke depan)
RANDOM_STATE = 42  # Seed untuk reproduksibilitas
N_JOBS = -1  # Menggunakan semua core CPU yang tersedia

## 2. Pemuatan dan Persiapan Data
Memuat data dari file Excel, mengatur indeks waktu, dan mengekstrak fitur waktu untuk digunakan sebagai variabel eksogen.

In [2]:
# Memuat data dari file Excel
data = pd.read_excel(
    'https://github.com/evanhfw/IEEE-code/raw/refs/heads/main/data/Malaysia.xlsx',
    parse_dates=['time'],
    index_col='time'
)

# Menghapus kolom 'No' dan baris terakhir, mengatur frekuensi per jam
data = data.drop(columns=['No']).iloc[:-1].asfreq('h')

# Mengganti nama kolom menjadi 'y' dan mengisi nilai yang hilang
data.columns = ['y']
data = data.ffill()

# Membuat fitur waktu
data['year'] = data.index.year
data['month'] = data.index.month
data['dayofyear'] = data.index.dayofyear
data['dayofmonth'] = data.index.day
data['dayofweek'] = data.index.dayofweek
data['hour'] = data.index.hour

# Memisahkan fitur (X) dan target (y)
X = data.drop(columns=['y'])
y = data['y']

# Menampilkan informasi dasar tentang data
print("Dimensi data:", data.shape)
print("\n5 baris pertama data:")
print(data.head())

Dimensi data: (17519, 7)

5 baris pertama data:
                           y  year  month  dayofyear  dayofmonth  dayofweek  \
time                                                                          
2009-01-01 01:00:00  30360.0  2009      1          1           1          3   
2009-01-01 02:00:00  29155.0  2009      1          1           1          3   
2009-01-01 03:00:00  28086.0  2009      1          1           1          3   
2009-01-01 04:00:00  28031.0  2009      1          1           1          3   
2009-01-01 05:00:00  27730.0  2009      1          1           1          3   

                     hour  
time                       
2009-01-01 01:00:00     1  
2009-01-01 02:00:00     2  
2009-01-01 03:00:00     3  
2009-01-01 04:00:00     4  
2009-01-01 05:00:00     5  


## 3. Pembuatan Fold untuk Validasi Silang
Membagi data menjadi fold untuk validasi silang berbasis waktu dengan jendela pelatihan dan pengujian.

In [3]:
# Membuat fold untuk validasi silang
fold = []
for i in range(RANGE_FOLD):
    fold.append((
        X.iloc[i*24:16775+i*24],           # X_train
        X.iloc[16775+i*24:16775+i*24+24],  # X_test
        y.iloc[i*24:16775+i*24],           # y_train
        y.iloc[16775+i*24:16775+i*24+24]   # y_test
    ))

# Mengambil fold pertama untuk contoh
X_train, X_test, y_train, y_test = fold[0]

# Menampilkan ukuran data untuk fold pertama
print("Ukuran X_train:", len(X_train))
print("Ukuran X_test:", len(X_test))
print("Ukuran y_train:", len(y_train))
print("Ukuran y_test:", len(y_test))

Ukuran X_train: 16775
Ukuran X_test: 24
Ukuran y_train: 16775
Ukuran y_test: 24


## 4. Inisialisasi Model dan Forecasting Horizon
Mengatur model XGBoost dengan `n_jobs=-1` dan forecasting horizon untuk prakiraan.

In [4]:
# Inisialisasi model XGBoost dengan n_jobs=-1
model = xgb.XGBRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS)

# Membuat forecasting horizon untuk 24 jam ke depan
fh = ForecastingHorizon(np.arange(1, FORECAST_HORIZON + 1))

# Inisialisasi forecaster sktime dengan panjang jendela
direct = DirectTabularRegressionForecaster(model, window_length=WINDOW_LENGTH)
dirrec = DirRecTabularRegressionForecaster(model, window_length=WINDOW_LENGTH)
multi = MultioutputTabularRegressionForecaster(model, window_length=WINDOW_LENGTH)
recursive = RecursiveTabularRegressionForecaster(model, window_length=WINDOW_LENGTH)

## 5. Fungsi untuk Prediksi dan Evaluasi
Fungsi untuk melakukan prakiraan dan menyimpan hasilnya ke file CSV.

In [5]:
def model_predict(
    model: DirectTabularRegressionForecaster
    | DirRecTabularRegressionForecaster
    | MultioutputTabularRegressionForecaster
    | RecursiveTabularRegressionForecaster,
    fold: list[tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]],
    fh: ForecastingHorizon,
    name: str,
    exogeneous: bool,
):
    """
    Melakukan prakiraan time series menggunakan validasi silang dengan model sktime.

    Parameter
    ---------
    model : sktime forecaster
        Model prakiraan (Direct, DirRec, Multioutput, atau Recursive).
    fold : list[tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]]
        Daftar fold untuk validasi silang berbasis waktu.
    fh : ForecastingHorizon
        Horizon prakiraan yang menentukan jumlah langkah ke depan.
    name : str
        Nama model untuk penamaan file output.
    exogeneous : bool
        Apakah menggunakan fitur eksogen (True) atau tidak (False).

    Side Effects
    ------------
    - Menyimpan prediksi dan metrik MAPE ke direktori 'predictions'.
    - Menampilkan progres pemrosesan setiap fold.
    """
    predictions_df = pd.DataFrame()
    test_df = pd.DataFrame()
    start_time = time.time()

    for i, (X_train, X_test, y_train, y_test) in enumerate(fold):
        print(f"Memproses fold {i + 1} - {name}")
        
        # Membuat direktori predictions jika belum ada
        if not os.path.exists("predictions"):
            os.makedirs("predictions")
            print("Direktori 'predictions' dibuat")

        if exogeneous:
            # Pelatihan dan prediksi dengan fitur eksogen
            model.fit(y_train, X=X_train, fh=fh)
            predictions_df[f"Fold {i + 1}"] = model.predict(fh, X=X_test).values
            test_df[f"Fold {i + 1}"] = y_test.values

            # Menghitung MAPE
            mape_df = np.abs(test_df - predictions_df) / test_df * 100

            # Menyimpan hasil
            predictions_df.to_csv(f"predictions/{name}_predictions_exo.csv")
            mape_df.to_csv(f"predictions/{name}_mape_exo.csv")
        else:
            # Pelatihan dan prediksi tanpa fitur eksogen
            model.fit(y_train, fh=fh)
            predictions_df[f"Fold {i + 1}"] = model.predict(fh).values
            test_df[f"Fold {i + 1}"] = y_test.values

            # Menghitung MAPE
            mape_df = np.abs(test_df - predictions_df) / test_df * 100

            # Menyimpan hasil
            predictions_df.to_csv(f"predictions/{name}_predictions.csv")
            mape_df.to_csv(f"predictions/{name}_mape.csv")

        print(f"Waktu pemrosesan fold {i + 1}: {time.time() - start_time:.2f} detik")

## 6. Evaluasi Model Sktime
Melakukan evaluasi untuk semua strategi prakiraan dengan dan tanpa fitur eksogen.

In [ ]:
print("=== EVALUASI MODEL SKTIME ===")

# 1. Direct Tabular Regression Forecaster
print("\n1. Direct Tabular Regression Forecaster")
print("   - Strategi: Prakiraan langsung untuk setiap langkah")
print("   - Kelebihan: Tidak ada propagasi kesalahan")
print("   - Kekurangan: Membutuhkan banyak model")

print("\nMenjalankan model Direct dengan fitur eksogen...")
model_predict(direct, fold, fh, "XGBoost_Direct_Exo", exogeneous=True)

print("\nMenjalankan model Direct tanpa fitur eksogen...")
model_predict(direct, fold, fh, "XGBoost_Direct_NoExo", exogeneous=False)

# 2. DirRec Tabular Regression Forecaster
print("\n2. DirRec Tabular Regression Forecaster")
print("   - Strategi: Kombinasi langsung dan rekursif")
print("   - Kelebihan: Menggabungkan keunggulan kedua metode")
print("   - Kekurangan: Kompleksitas lebih tinggi")

print("\nMenjalankan model DirRec dengan fitur eksogen...")
model_predict(dirrec, fold, fh, "XGBoost_DirRec_Exo", exogeneous=True)

print("\nMenjalankan model DirRec tanpa fitur eksogen...")
model_predict(dirrec, fold, fh, "XGBoost_DirRec_NoExo", exogeneous=False)

# 3. Recursive Tabular Regression Forecaster
print("\n3. Recursive Tabular Regression Forecaster")
print("   - Strategi: Menggunakan prediksi sebelumnya sebagai input")
print("   - Kelebihan: Menangkap pola temporal")
print("   - Kekurangan: Propagasi kesalahan")

print("\nMenjalankan model Recursive dengan fitur eksogen...")
model_predict(recursive, fold, fh, "XGBoost_Recursive_Exo", exogeneous=True)

print("\nMenjalankan model Recursive tanpa fitur eksogen...")
model_predict(recursive, fold, fh, "XGBoost_Recursive_NoExo", exogeneous=False)

# 4. Multioutput Tabular Regression Forecaster
print("\n4. Multioutput Tabular Regression Forecaster")
print("   - Strategi: Memprediksi semua langkah sekaligus")
print("   - Kelebihan: Menangkap dependensi antar langkah")
print("   - Kekurangan: Membutuhkan model yang mendukung multi-output")

print("\nMenjalankan model Multioutput dengan fitur eksogen...")
model_predict(multi, fold, fh, "XGBoost_Multioutput_Exo", exogeneous=True)

print("\nMenjalankan model Multioutput tanpa fitur eksogen...")
model_predict(multi, fold, fh, "XGBoost_Multioutput_NoExo", exogeneous=False)

print("\nSemua evaluasi model selesai!")
print("Hasil disimpan di direktori 'predictions'.")

=== EVALUASI MODEL SKTIME ===

1. Direct Tabular Regression Forecaster
   - Strategi: Prakiraan langsung untuk setiap langkah
   - Kelebihan: Tidak ada propagasi kesalahan
   - Kekurangan: Membutuhkan banyak model

Menjalankan model Direct dengan fitur eksogen...
Memproses fold 1 - XGBoost_Direct_Exo
Waktu pemrosesan fold 1: 261.81 detik
Memproses fold 2 - XGBoost_Direct_Exo
Waktu pemrosesan fold 2: 522.02 detik

Menjalankan model Direct tanpa fitur eksogen...
Memproses fold 1 - XGBoost_Direct_NoExo
Waktu pemrosesan fold 1: 71.88 detik
Memproses fold 2 - XGBoost_Direct_NoExo
Waktu pemrosesan fold 2: 148.25 detik

2. DirRec Tabular Regression Forecaster
   - Strategi: Kombinasi langsung dan rekursif
   - Kelebihan: Menggabungkan keunggulan kedua metode
   - Kekurangan: Kompleksitas lebih tinggi

Menjalankan model DirRec dengan fitur eksogen...
Memproses fold 1 - XGBoost_DirRec_Exo
Waktu pemrosesan fold 1: 74.87 detik
Memproses fold 2 - XGBoost_DirRec_Exo
Waktu pemrosesan fold 2: 149.67 

## 7. Model XGBoost Sederhana
Membuat model XGBoost sederhana tanpa pendekatan sktime untuk perbandingan.

In [8]:
# Inisialisasi dan pelatihan model XGBoost sederhana dengan n_jobs=-1
print("\n=== MODEL XGBOOST SEDERHANA ===")
print("Melatih model XGBoost sederhana...")
simple_model = xgb.XGBRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS)
start_time = time.time()
simple_model.fit(X_train, y_train)
print(f"Waktu pelatihan: {time.time() - start_time:.2f} detik")

# Membuat prediksi
print("Membuat prediksi...")
predictions = simple_model.predict(X_test)

# Menghitung metrik evaluasi
mape = np.abs(y_test - predictions) / y_test * 100
mae = np.abs(y_test - predictions).mean()
rmse = np.sqrt(((y_test - predictions) ** 2).mean())

# Menampilkan hasilLoading
print(f"\n=== HASIL EVALUASI ===")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape.mean():.4f}%")
print(f"Rentang MAPE: {mape.min():.4f}% - {mape.max():.4f}%")

# Membuat DataFrame hasil
results_df = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": predictions,
    "Error": y_test.values - predictions,
    "MAPE": mape.values,
})

# Menampilkan 10 baris pertama hasil
print("\n=== CONTOH PREDIKSI ===")
print(results_df.head(10))

# Menyimpan hasil
if not os.path.exists("predictions"):
    os.makedirs("predictions")
results_df.to_csv("predictions/simple_xgboost_results.csv")
print("\nHasil disimpan di: predictions/simple_xgboost_results.csv")


=== MODEL XGBOOST SEDERHANA ===
Melatih model XGBoost sederhana...
Waktu pelatihan: 0.13 detik
Membuat prediksi...

=== HASIL EVALUASI ===
Mean Absolute Error (MAE): 1208.0827
Root Mean Square Error (RMSE): 1618.3115
Mean Absolute Percentage Error (MAPE): 2.4749%
Rentang MAPE: 0.0522% - 7.0632%

=== CONTOH PREDIKSI ===
    Actual     Predicted        Error      MAPE
0  33142.0  33606.738281  -464.738281  1.402264
1  30967.0  31468.718750  -501.718750  1.620172
2  29641.0  30880.341797 -1239.341797  4.181174
3  29039.0  28518.164062   520.835938  1.793574
4  28254.0  28026.625000   227.375000  0.804753
5  28395.0  28505.246094  -110.246094  0.388259
6  28090.0  28418.234375  -328.234375  1.168510
7  30411.0  31383.900391  -972.900391  3.199173
8  38878.0  40232.667969 -1354.667969  3.484408
9  47696.0  49153.382812 -1457.382812  3.055566

Hasil disimpan di: predictions/simple_xgboost_results.csv
